In [ ]:
import os
import json
from datetime import datetime

## Define input and output filepaths

In [ ]:
MATTERPORT_DIR = "/media/rsl_admin/T7/matterport/data/v1/scans"

LATTICE_GRAPHS_DIR = "/media/rsl_admin/T7/matterport/lattice_graphs"

In [ ]:
OPENSCENE_REGION_SEGS_DIR = \
    "/home/rsl_admin/openscene/comparison_outputs/segmentations/region-openseg-matterport-test/"


In [ ]:
OUTPUT_DIR = "/home/rsl_admin/openscene/comparison_outputs/eval_outputs"

save_dir = os.path.join(OUTPUT_DIR, f"region-{datetime.now().strftime('%Y-%m-%d_%H-%M')}")
os.makedirs(save_dir, exist_ok=True)

print(f"saving outputs to: {save_dir}")

## Figure out which scans to process

In [ ]:
scans = set()
for fname in os.listdir(OPENSCENE_REGION_SEGS_DIR):
    scans.add(fname.split("_")[0])
    
scans = tuple(sorted(scans))

print(scans)

## Evaluation parameters

In [ ]:
BLACKLISTED_REGION_LABELS = (
    "other room",

    # no appropriate tag
    "dining booth",
    "entryway/foyer/lobby",
    "outdoor",
)

In [ ]:
params = {
        "label_params": {
            "blacklisted_labels": BLACKLISTED_REGION_LABELS,
        },
    
        # Make sure sides of proposal boxes are at least this long
        "min_proposal_box_length": 0.1,
    
    
        # From email with the author of OVIR
        "cluster_min_points": 50,

        
        "dbscan_params": {
            "eps": 0.5,
            "min_points": 1,
            "print_progress": False,
        },
    }

## Run evaluation on the scans in the segmentations directory

In [ ]:
from evaluate_matterport_scan_region import evaluate_matterport_scan_region_localizations

In [ ]:
# save the params
params_save_path = os.path.join(save_dir, 'params.json')
with open(params_save_path, 'w') as f:
    json.dump(params, f)


# run the evaluation
for scan_name in scans:

    scan_dir = os.path.join(MATTERPORT_DIR, f"{scan_name}/{scan_name}")
    lattice_graph_path = os.path.join(LATTICE_GRAPHS_DIR, f"{scan_name}_lattice_graph.pkl")
    
    evaluate_matterport_scan_region_localizations(
        params=params,
        scan_dir=scan_dir,
        openscene_region_segs_dir=OPENSCENE_REGION_SEGS_DIR,
        lattice_graph_path=lattice_graph_path,
        output_dir=save_dir
    )